### Shared Word

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
lang_folder = "Italian"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language
lang_pair = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language
word_start = 0  # 0
word_end = 28  # 28 

In [4]:
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}").mkdir(parents=True, exist_ok=True)

In [5]:
def remove_repetition(word_group):
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [6]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
331836,polygamisch,5
331837,shiit,5
331838,ambient,5
331839,zinswendingen,5


In [7]:
df_word = df_word_all.iloc[word_start:word_end,]
df_word

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
5,is,12813896
6,een,11124670
7,niet,10747590
8,en,8257870
9,van,6532887


In [8]:
lang_pair_list = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_And_{lang_pair.lower().capitalize()}*_All.xlsx")
lang_pair_list

['/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Dutch/Dutch_And_Turkish (tur)_Etymologeek_Result_All.xlsx']

In [9]:
df_pair = pd.read_excel(f"{lang_pair_list[0]}")
df_pair

,dict_entry_main,turkish_word
0,checken,kontrol
1,cheque,kontrol
2,chequeverkeer,kontrol
3,controle,kontrol
4,broche,broş
...,...,...
288,minaret,minare
289,odalisk,odalık
290,tulband,tülbent
291,tulp,tülbent


In [10]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique","bikini","penis"]

In [11]:
word_select = df_word["word"].values.tolist()

In [12]:
words = df_pair["dict_entry_main"].values.tolist()

In [13]:
word_select_set = set(word_select)
disable_word_set = set(disable_list)
words_set = set(words)

In [14]:
word_list = list(word_select_set.union(words_set.difference(disable_word_set)))

In [15]:
# 2 way  Not: 1 way is long and in Shared Word ReDe part 
data_kind_list = ["Two", "Three", "Four", "Five", "Sentence"]
for i in data_kind_list:
    sent_ngram_file = f"{i}" # Two, Three, Four, Five, Sentence
 
    if sent_ngram_file.lower() != "sentence":
        data_kind = f"{sent_ngram_file}gram"  # Twogram, Threegram, Fourgram, Fivegram
        folder_path = "N Gram"
    else:
        data_kind = "sentence"
        folder_path = "Sentence"

    df = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/{folder_path.capitalize()}/Merge/{data_kind.capitalize()}_Merge.csv")
    d_list  = df.iloc[:,0].values.tolist()

    resultlist = []
    manager = multiprocessing.Manager()
    resultlist = manager.list()
    
    def word_in_wordgroup(d_list):
        mergelist = []
        try:
            word = d_list.split()
        except:
            word = []
            #pass  disabled for non split value
        var1 = range(len(word))
        for j in var1:
            if word[j] in word_list:
                mergelist.append(word[j])
                if len(mergelist) == len(word):
                        resultlist.append(d_list)
                            
    if __name__ == '__main__':
        # with Pool(16) as p:
        with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
            p.map(word_in_wordgroup, d_list) # string_word liste

    result_list = list(resultlist)
    df_result = pd.DataFrame(result_list, columns=[0])  # add columns parameter for empty result
    df_result = df_result.rename(columns = {0: f"{data_kind.lower()}"})
    df_merge = pd.merge(df_result, df, how="left", on=f"{data_kind.lower()}")
    df_merge_result = df_merge.sort_values(by="frequency", ascending=False)
    df_merge_result.drop_duplicates(inplace=True)
    df_merge_result[f"{data_kind.lower()}"] = df_merge_result[f"{data_kind.lower()}"].apply(lambda x: remove_repetition(x))  # added remove_repetition 
    df_merge_result = df_merge_result[df_merge_result[f"{data_kind.lower()}"] != "repetitive_word_group"]  # remove_repetition part
    df_merge_result.reset_index(drop=True, inplace=True)

    df_merge_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_{data_kind.capitalize()}.xlsx", index=False)
    

#### Concat Result

In [16]:
df_word = pd.DataFrame(word_list, columns=["word"])
df_word

,word
0,je
1,agorafobie
2,ananas
3,sultan
4,ik
...,...
311,bloes
312,stercatalogus
313,afzetmarkt
314,gorilla


In [17]:
df_shared_twogram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram.xlsx")
df_shared_twogram = df_shared_twogram.loc[:,["twogram","frequency"]]
df_shared_twogram.rename(columns={"frequency":"freq_twogram"}, inplace=True)
df_shared_twogram

,twogram,freq_twogram
0,ik heb,1113451
1,het is,1102152
2,dat is,939846
3,wat is,698111
4,heb je,691326
...,...,...
6006,me school,3
6007,het paradox,3
6008,het pandora,3
6009,mijnen met,3


In [18]:
df_shared_threegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Threegram.xlsx")
df_shared_threegram = df_shared_threegram.loc[:,["threegram","frequency"]]
df_shared_threegram.rename(columns={"frequency":"freq_threegram"}, inplace=True)
df_shared_threegram

,threegram,freq_threegram
0,wat is er,364876
1,het is een,135216
2,ik heb een,110642
3,ik heb het,107020
4,wat is dat,92873
...,...,...
23374,de coach hij,3
23375,de coach maar,3
23376,als er die,3
23377,de coalitie niet,3


In [19]:
df_shared_fourgram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fourgram.xlsx")
df_shared_fourgram = df_shared_fourgram.loc[:,["fourgram","frequency"]]
df_shared_fourgram.rename(columns={"frequency":"freq_fourgram"}, inplace=True)
df_shared_fourgram

,fourgram,freq_fourgram
0,wat is er met,43390
1,dat is het niet,14604
2,is er met je,12640
3,het is maar een,10266
4,dat heb ik niet,9969
...,...,...
30229,het is n paradox,3
30230,te zijn is ze,3
30231,te zijn is niet,3
30232,van de scholen te,3


In [20]:
df_shared_fivegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fivegram.xlsx")
df_shared_fivegram = df_shared_fivegram.loc[:,["fivegram","frequency"]]
df_shared_fivegram.rename(columns={"frequency":"freq_fivegram"}, inplace=True)
df_shared_fivegram

,fivegram,freq_fivegram
0,wat is er met je,12627
1,dat is niet wat ik,2548
2,het is niet wat je,2357
3,wat is er met de,1716
4,je dat ik dat niet,1503
...,...,...
14006,we we zijn in de,3
14007,we we zijn van de,3
14008,dat ze het in die,3
14009,te me dat ze er,3


In [21]:
df_shared_sentence = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Sentence.xlsx")
df_shared_sentence = df_shared_sentence.loc[:,["sentence","frequency"]]
df_shared_sentence.rename(columns={"frequency":"freq_sentence"}, inplace=True)
df_shared_sentence

,sentence,freq_sentence
0,wat is er,106497
1,wat is dat,62768
2,wat was dat,20127
3,wat is het,18734
4,dat is het,17533
...,...,...
4265,maar chef wat,3
4266,was je dat niet,3
4267,dokter het is een,3
4268,dokter is hij,3


In [22]:
df_shared_all = pd.concat([df_word, df_shared_twogram, df_shared_threegram, df_shared_fourgram, df_shared_fivegram, df_shared_sentence], axis=1)
df_shared_all

,word,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence
0,je,ik heb,1113451.0,wat is er,364876.0,wat is er met,43390,wat is er met je,12627.0,wat is er,106497.0
1,agorafobie,het is,1102152.0,het is een,135216.0,dat is het niet,14604,dat is niet wat ik,2548.0,wat is dat,62768.0
2,ananas,dat is,939846.0,ik heb een,110642.0,is er met je,12640,het is niet wat je,2357.0,wat was dat,20127.0
3,sultan,wat is,698111.0,ik heb het,107020.0,het is maar een,10266,wat is er met de,1716.0,wat is het,18734.0
4,ik,heb je,691326.0,wat is dat,92873.0,dat heb ik niet,9969,je dat ik dat niet,1503.0,dat is het,17533.0
...,...,...,...,...,...,...,...,...,...,...,...
30229,NaN,NaN,NaN,NaN,NaN,het is n paradox,3,NaN,NaN,NaN,NaN
30230,NaN,NaN,NaN,NaN,NaN,te zijn is ze,3,NaN,NaN,NaN,NaN
30231,NaN,NaN,NaN,NaN,NaN,te zijn is niet,3,NaN,NaN,NaN,NaN
30232,NaN,NaN,NaN,NaN,NaN,van de scholen te,3,NaN,NaN,NaN,NaN


In [23]:
df_shared_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency.xlsx", index=False)

In [24]:
def word_in_twogram(df, list_column, target_column):

    '''word_in_twogram(df, "word", "twogram"):
       df is dataframe, list_column and target_column are 
       dataframe column string name. list_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{list_column}"].dropna():
        try:
            word_in_twogram = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] 
        except:
            pass        
        word_in_twogram.insert(0,"word",i)
        df_word_result = pd.concat([df_word_result,word_in_twogram], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [25]:
df_word_order_twogram = word_in_twogram(df_shared_all, "word", "twogram")
df_word_order_threegram = word_in_twogram(df_shared_all, "word", "threegram") 
df_word_order_fourgram = word_in_twogram(df_shared_all, "word", "fourgram") 
df_word_order_fivegram = word_in_twogram(df_shared_all, "word", "fivegram")
df_word_order_sentence = word_in_twogram(df_shared_all, "word", "sentence") 

In [26]:
df_word_order_join_twogram = df_word_order_twogram.groupby(["word"])["twogram"].apply(", ".join).reset_index()   # df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].transform(lambda x: ','.join(x))
df_word_order_join_threegram = df_word_order_threegram.groupby(["word"])["threegram"].apply(", ".join).reset_index()
df_word_order_join_fourgram = df_word_order_fourgram.groupby(["word"])["fourgram"].apply(", ".join).reset_index()
df_word_order_join_fivegram = df_word_order_fivegram.groupby(["word"])["fivegram"].apply(", ".join).reset_index()
df_word_order_join_sentence = df_word_order_sentence.groupby(["word"])["sentence"].apply(", ".join).reset_index()

In [27]:
dfs = [df_word_order_join_twogram,df_word_order_join_threegram,df_word_order_join_fourgram,df_word_order_join_fivegram,df_word_order_join_sentence]

In [28]:
df_word_order_join_all = reduce(lambda  left,right: pd.merge(left,right, on=['word'], how='outer'), dfs)  # left,right make left to right merge
#df_word_order_join_all = reduce(lambda  right,left: pd.merge(left,right, on=['word'], how='outer'), dfs)  # right,left make right to left merge
df_word_order_join_all  

,word,twogram,threegram,fourgram,fivegram,sentence
0,acrobaat,"acrobaat met, de acrobaat, acrobaat in, als ac...","als acrobaat in, dat ik acrobaat, n acrobaat m...",NaN,NaN,NaN
1,actrice,"een actrice, de actrice, die actrice, je actri...","ze is actrice, is een actrice, je een actrice,...","ze is een actrice, ze was een actrice, dat je ...","die is van die actrice, je dat ik n actrice, d...","een actrice, de actrice, ze is actrice, ze is ..."
2,advocatuur,"de advocatuur, is advocatuur","met de advocatuur, is de advocatuur, van de ad...",hij is de advocatuur,NaN,het is advocatuur
3,agent,"een agent, is agent, de agent, die agent, n ag...","is een agent, je een agent, was een agent, van...","hij is een agent, er is een agent, het is een ...","dat het een agent was, dat je een agent was, d...","een agent, hij is een agent, de agent, die age..."
4,agente,"een agente, die agente, is agente, agente die,...","ze is agente, is een agente, een agente van, d...","een agente van de, dat ze agente was, je met d...","ze dat ze agente was, heb je met die agente, h...","ze is agente, een agente, die agente, je agent..."
...,...,...,...,...,...,...
271,yoghurt,"en yoghurt, yoghurt en, de yoghurt, die yoghur...","yoghurt maar mayonaise, yoghurt is niet, die y...","die yoghurt is niet, is je yoghurt op, het is ...",dat het chocolade yoghurt was,"en yoghurt, is je yoghurt op, het is maar yogh..."
272,zalm,"de zalm, zalm en, een zalm, zalm is, zalm met,...","van de zalm, met de zalm, zalm die ik, en de z...","zalm die ik heb, er een zalm in, ik heb wat za...","er wat met de zalm, zijn als je de zalm","de zalm, op zalm, het is zalm, is er wat met d..."
273,ze,"ze is, ze zijn, dat ze, is ze, ze niet, zijn z...","ik heb ze, ze is een, ze is niet, je dat ze, n...","ze is er niet, ik heb ze niet, dat is wat ze, ...","wat is er met ze, en als ze dat niet, wat heb ...","ze zijn er, dat is ze, ze is, ik heb ze, ze is..."
274,zijn,"we zijn, ze zijn, er zijn, zijn we, zijn er, z...","we zijn er, we zijn niet, ze zijn er, we zijn ...","we zijn in de, wat zijn dat voor, dat zijn we ...","we zijn van de politie, wat is er met zijn, we...","we zijn er, wat zijn dat, ze zijn er, dat zijn..."


In [29]:
df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Result_Without_Frequency.xlsx", index=False)

#### Copy Move And Delete

In [30]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared*.xlsx")
output_file

['Dutch_Turkish_Shared_Twogram.xlsx',
 'Dutch_Turkish_Shared_Threegram.xlsx',
 'Dutch_Turkish_Shared_Fourgram.xlsx',
 'Dutch_Turkish_Shared_Fivegram.xlsx',
 'Dutch_Turkish_Shared_Sentence.xlsx',
 'Dutch_Turkish_Shared_Result_With_Frequency.xlsx',
 'Dutch_Turkish_Shared_Join_Result_Without_Frequency.xlsx']

In [31]:
for k in output_file:
    source = k # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [32]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass